In [1]:
import pandas as pd
from utilities.api_functions import *

# load the lookup table
geoid_lu_df = pd.read_csv('/Users/joshpaul/epa-justice/repo/epa-justice/tbl/NCRPlaces_Census_04192024.csv')

# to test, limit the lookup table to a few locations to test each geography type
# we include an individual county, incorporated place, census designated place, 
# and a 1-to-many entry (Eagle River) that has multiple census tracts per single GVV ID
geoid_lu_df_test = geoid_lu_df[geoid_lu_df['name'].isin(['Eagle River', 'Fairbanks', 'Arctic Village', 'Fort Yukon'])]

In [2]:
geoid_lu_df_test

,id,name,alt_name,region,country,latitude,longitude,type,GEOIDFQ,PLACENAME,AREATYPE,COMMENT
10,AK124,Fairbanks,NaN,Alaska,US,64.8378,-147.716,community,0500000US02090,Fairbanks North Star Borough,County,Data represent information from nearest [COLUM...
45,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000201,Census Tract 2.01,Census tract,Data for this place represent multiple merged ...
46,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000202,Census Tract 2.02,Census tract,Data for this place represent multiple merged ...
47,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000204,Census Tract 2.04,Census tract,Data for this place represent multiple merged ...
48,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000205,Census Tract 2.05,Census tract,Data for this place represent multiple merged ...
49,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000206,Census Tract 2.06,Census tract,Data for this place represent multiple merged ...
77,AK22,Arctic Village,Vashrąįį K'ǫǫ,Alaska,US,68.1269,-145.538,community,1600000US0203990,Arctic Village CDP,Census designated place,Data represent information from nearest [COLUM...
166,AK130,Fort Yukon,Gwichyaa Zheh,Alaska,US,66.5647,-145.274,community,1600000US0226760,Fort Yukon city,Incorporated place,Data represent information from nearest [COLUM...


In [3]:
# use the individual fetch functions get data for GVV IDs in the test lookup table
# follow the printed URLs to QC individual values from the results table
for gvv_id in list(geoid_lu_df_test.id.unique()):
    dhc = fetch_census_data_and_compute("dhc", gvv_id, geoid_lu_df_test, print_url=True)
    print(dhc)
    acs5 = fetch_census_data_and_compute("acs5", gvv_id, geoid_lu_df_test, print_url=True)
    print(acs5)
    cdc = fetch_cdc_data_and_compute(gvv_id, geoid_lu_df_test, print_url=True)
    print(cdc)


Requesting US Census data from: https://api.census.gov/data/2020/dec/dhc?get=P12_001N,P12_002N,P12_026N,P12_003N,P12_004N,P12_005N,P12_006N,P12_020N,P12_021N,P12_022N,P12_023N,P12_024N,P12_025N,P12_027N,P12_028N,P12_029N,P12_030N,P12_044N,P12_045N,P12_046N,P12_047N,P12_048N,P12_049N&for=county:090&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706
  GEOID  total_population  pct_65_plus  pct_under_18
0   090           95655.0        11.46         23.95
Requesting US Census data from: https://api.census.gov/data/2020/acs/acs5/subject?get=S1810_C03_001E,S1810_C03_001M,S2701_C03_001E,S2701_C03_001M,S2701_C05_001E,S2701_C05_001M&for=county:090&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706
   pct_w_disability  moe_pct_w_disability  pct_insured  moe_pct_insured  \
0              11.1                   1.0         90.6              1.0   

   pct_uninsured  moe_pct_uninsured GEOID  
0            9.4                1.0   090  
Requesting CDC data from: https://data.cdc.gov/reso

In [4]:
# if test results look good, run the fetch and merge with the full lookup table and concatenate all results
# this will take a while (potentially 60 minutes for the entire dataset), as it makes many calls to the various APIs
# heavy API usage may be "throttled" by the service
results = []
for gvv_id in list(geoid_lu_df.id.unique()):
    results.append(fetch_and_merge(geoid_lu_df, gvv_id))
results_df = pd.concat(results)
results_df

,id,name,areatype,placename,GEOID,total_population,pct_65_plus,pct_under_18,pct_w_disability,moe_pct_w_disability,...,pct_asthma,pct_copd,pct_hd,pct_stroke,pct_diabetes,pct_kd,pct_minority,pct_no_hsdiploma,pct_below_150pov,pct_no_bband
0,BORO1,Aleutians East Borough,County,Aleutians East Borough,013,3420.0,6.87,8.77,10.7,2.2,...,7.8,5.4,5.7,3.3,13.1,3.0,87.2,15.3,22.7,42.5
0,BORO19,Municipality of Anchorage,County,Anchorage Municipality,020,291247.0,12.41,23.46,11.2,0.6,...,9.4,5.3,4.7,2.6,7.9,2.6,43.9,5.8,15.1,7.3
0,AK15,Anchorage,County,Anchorage Municipality,020,291247.0,12.41,23.46,11.2,0.6,...,9.4,5.3,4.7,2.6,7.9,2.6,43.9,5.8,15.1,7.3
0,CENS9,Bethel Census Area,County,Bethel Census Area,050,18666.0,8.24,35.02,11.4,1.0,...,12.8,10.4,7.9,4.8,14.8,3.9,90.8,18.0,43.9,25.2
0,BORO17,Bristol Bay Borough,County,Bristol Bay Borough,060,844.0,16.11,22.04,15.9,4.1,...,10.0,6.9,6.9,3.6,10.7,3.4,58.8,5.3,8.0,23.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,AK430,Willow,Census designated place,Willow CDP,85280,2196.0,26.37,15.94,12.9,3.4,...,9.9,8.8,6.8,3.5,10.1,3.2,12.2,7.3,26.1,24.5
0,AK556,Willow Creek,Census designated place,Willow Creek CDP,85290,190.0,NaN,NaN,89.2,15.0,...,9.7,8.8,7.2,3.6,11.5,3.3,13.9,6.3,NaN,NaN
0,AK431,Wiseman,Census designated place,Wiseman CDP,85610,5.0,NaN,NaN,66.7,26.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AK432,Womens Bay,Census designated place,Womens Bay CDP,85680,743.0,NaN,23.96,7.6,6.4,...,10.5,8.0,5.5,2.9,8.9,2.6,27.9,5.0,8.6,NaN


In [5]:
# save to CSV
results_df.to_csv('tbl/data_to_export.csv', index=False)